In [ ]:
# ========================================================
# Construção de Índice de Vulnerabilidade à Seca
# Exemplo para Google Colab
# Guilherme A. L. Torres
# ========================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------------------------------------
# Exemplo de dataset fictício
# Cada linha = município
# Variáveis climáticas: precipitação anual, SPI médio
# Variáveis sociais: IDH, % dependência agrícola

data = {
    "Municipio": ["A", "B", "C", "D", "E"],
    "Precipitacao": [850, 1200, 600, 700, 950],   # mm/ano
    "SPI": [-1.2, 0.1, -2.0, -0.5, -0.8],         # índice de precipitação padronizado
    "IDH": [0.65, 0.72, 0.55, 0.60, 0.68],        # índice de desenvolvimento humano
    "Depend_Agric": [0.70, 0.40, 0.80, 0.65, 0.55] # % população dependente da agricultura
}

df = pd.DataFrame(data)

# --------------------------------------------------------
# Normalização Min-Max (0 a 1)
def min_max_norm(series):
    return (series - series.min()) / (series.max() - series.min())

df_norm = df.copy()
df_norm["Precipitacao"] = min_max_norm(df["Precipitacao"])
df_norm["SPI"] = min_max_norm(df["SPI"])
df_norm["IDH"] = min_max_norm(df["IDH"])
df_norm["Depend_Agric"] = min_max_norm(df["Depend_Agric"])

# --------------------------------------------------------
# Definição dos pesos (aqui iguais, mas podem vir de PCA)
pesos = {
    "Precipitacao": 0.25,
    "SPI": 0.25,
    "IDH": 0.25,
    "Depend_Agric": 0.25
}

# Cálculo do índice composto (quanto maior, maior vulnerabilidade)
df_norm["Indice_Vulnerabilidade"] = (
    df_norm["Precipitacao"] * pesos["Precipitacao"] +
    df_norm["SPI"] * pesos["SPI"] +
    (1 - df_norm["IDH"]) * pesos["IDH"] +    # inversão: IDH alto = menor vulnerabilidade
    df_norm["Depend_Agric"] * pesos["Depend_Agric"]
)

# --------------------------------------------------------
# Visualização
plt.figure(figsize=(10,5))
sns.barplot(x="Municipio", y="Indice_Vulnerabilidade", data=df_norm, palette="Reds")
plt.title("Índice de Vulnerabilidade à Seca (0-1)")
plt.ylabel("Vulnerabilidade")
plt.xlabel("Município")
plt.ylim(0,1)
plt.show()

# --------------------------------------------------------
# Resultado final
print("Índice de Vulnerabilidade Calculado:")
print(df_norm[["Municipio", "Indice_Vulnerabilidade"]].sort_values(by="Indice_Vulnerabilidade", ascending=False))
